Import packages and processed run metrics

In [10]:
# packages
import re
import numpy as np
import pandas as pd

# control
project_names = [
    'decay--sst2',
    'decay--pubmed',
    'decay--cifar--alpha5.0',
    'decay--cifar--alpha0.5',
    'decay--cifar--alpha0.1'
]
validation_metric = 'Results/val_acc'

# read each data set
df = []
print('input datasets:')
for project_name in project_names:
    temp = pd.read_csv(f'{project_name}.csv')
    print('\t', temp.shape)
    df.append(temp)

df = pd.concat(df, axis=0, ignore_index=True)
print('all runs:', df.shape)

input datasets:
	 (1200, 35)
	 (780, 28)
	 (520, 27)
	 (520, 27)
	 (520, 27)
all runs: (3540, 36)


Write function to return summaries of run's metric performance

In [7]:
def process_run_metrics(filtered_df, method='exact'):
    pass


    ## Process metrics and get top runs for each
    # are large or small metric values are desirable?
    descending_metrics = [name for name in filtered_df.columns if re.search('Results.*test', name)]
    ascending_metrics = [
        descending_metrics.pop(descending_metrics.index(name))
        for name in descending_metrics
        if re.search('std', name)
    ]
    print('metrics:')
    print('\t', descending_metrics)
    print('\t', ascending_metrics)

    # convert metrics to numeric
    #filtered_df[descending_metrics + ascending_metrics] = filtered_df[descending_metrics + ascending_metrics].apply(pd.to_numeric)

    # all non-metrics columns are used to identify the experimental run
    filtered_runs = filtered_df[[
        name for name in filtered_df.columns
        if name not in descending_metrics + ascending_metrics
    ]]

    # rank the metrics
    ranked_descending = filtered_df.groupby(['dataset'])[descending_metrics].rank(
        method='min',
        ascending=False
    )
    ranked_ascending = filtered_df.groupby(['dataset'])[ascending_metrics].rank(
        method='min',
        ascending=True
    )

    # combine and sort the ranked_metrics
    ranked_metrics = pd.concat([ranked_descending, ranked_ascending], axis=1)
    ranked_metrics = ranked_metrics[sorted(ranked_metrics.columns)]
    filtered_ranks = filtered_runs.join(ranked_metrics)


    ## Manipulate rank data to be summarized by runs and metrics
    # convert to long format
    # filter to top ranks
    long_filtered_ranks = pd.melt(filtered_ranks, id_vars=filtered_runs.columns, var_name='metric')
    top_filtered_metrics = long_filtered_ranks.loc[long_filtered_ranks.value <= 3].copy()  # top 3 runs

    # compute rank summaries to understand what runs are top overall
    top_filtered_metrics['rank_one_ind'] = (top_filtered_metrics.value == 1)
    top_filtered_metrics['rank_two_ind'] = (top_filtered_metrics.value == 2)
    top_filtered_metrics['rank_three_ind'] = (top_filtered_metrics.value == 3)
    top_filtered_metrics.replace(False, pd.NA, inplace=True)

    # summarized metric ranks for run type
    rank_summary_columns = ['rank_one_ind', 'rank_two_ind', 'rank_three_ind', 'value']
    #id_columns = ['dataset', 'method', 'finetune']
    id_columns = ['method', 'finetune']
    run_summary = top_filtered_metrics.groupby(id_columns)[rank_summary_columns].count()

    # summarize metric ranks for metric choice
    metric_summary = top_filtered_metrics.loc[top_filtered_metrics.method == method]
    metric_summary = metric_summary.sort_values(by='metric').groupby('metric')[rank_summary_columns].count()

    return (
        run_summary,
        metric_summary,
        filtered_runs
    )


Filter data to runs of interest
Apply summary function to filtered dataset

In [12]:
df.K

0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
3535   NaN
3536   NaN
3537   NaN
3538   NaN
3539   NaN
Name: K, Length: 3540, dtype: float64

In [9]:
subset_df = df
print('input dataset:', df.shape)

## row (run) filtering
# remove extra hyper-parameter searches
subset_df = subset_df.loc[(df.n_epochs < 6)]
print('\t reduce to 3 or fewer local update steps:', df.shape)
subset_df = subset_df.loc[(df.K < 6) | (df.method != 'pfedme')]
print('\t reduce to 3 or fewer local meta-learning steps for pfedme:', df.shape)
subset_df = subset_df.loc[(int(10 * df.beta) % 2 == 0) | (df.method != 'exact')]
print('\t reduce beta grid for decay:', df.shape)

"""
subset_df = subset_df.loc[[
    True if beta in betas else False
    for beta in subset_df.beta
    ] | (subset_df.method != 'exact')
]

# column (metric) filtering
subset_df = subset_df[[
    name for name in subset_df.columns
    if not re.search('f1', name)
       and not re.search('loss', name)
]]
"""

input dataset: (3540, 36)
	 reduce to 3 or fewer local update steps: (3540, 36)
	 reduce to 3 or fewer local meta-learning steps for pfedme: (3540, 36)


TypeError: cannot convert the series to <class 'int'>

In [ ]:
## get best runs for each group
# regardless of finetuning
best_runs = df.loc[
    df.groupby(['dataset', 'method', 'finetune']) \
    [validation_metric].idxmax()  # return index of max validation metric
]

# treat finetuning groups as seperate
ft_yes = best_runs.loc[best_runs.finetune == 1]
ft_no = best_runs.loc[best_runs.finetune == 0]

For all best runs, regardless of finetuning, produce summaries

In [ ]:
(all_run_summary, all_metric_summary, _) = process_run_metrics(best_runs)
print('all runs:', all_run_summary.shape)
print('all metrics:', all_metric_summary.shape)

# run summary
all_run_summary.sort_values(by='value', ascending=False)

View processed summaries

In [ ]:
# metrics summary
sort_by = ['value', 'rank_one_ind', 'rank_two_ind', 'rank_three_ind']
all_metric_summary.sort_values(by=sort_by, ascending=False)

Now seperately, based on finetuning status, repeat the above summary

In [ ]:
(yes_run_summary, yes_metric_summary, _) = process_run_metrics(ft_yes)
print('ft_yes runs:', yes_run_summary.shape)
print('ft_yes metrics:', yes_metric_summary.shape)
(no_run_summary, no_metric_summary, _) = process_run_metrics(ft_no)
print('ft_no runs:', all_run_summary.shape)
print('ft_no metrics:', all_metric_summary.shape)

In [ ]:
# run summary
yes_run_summary.sort_values(by='value', ascending=False)

In [ ]:
# metrics summary
yes_metric_summary.sort_values(by=sort_by, ascending=False)

In [ ]:
# run summary
no_run_summary.sort_values(by='value', ascending=False)

In [ ]:
# metrics summary
no_metric_summary.sort_values(by=sort_by, ascending=False)